# 📗 Part 3 v3 완전판\n\n**Priority 1:** Ensemble+Statistical Test, n_iter=200  \n**Priority 2:** Cumulative Gains, 시각화, CV 임계값  \n**Priority 3:** Winsorizer, Calibration, Learning Curve

## 0. 환경 설정

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import platform
if platform.system() == 'Darwin': plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': plt.rc('font', family='Malgun Gothic')
else: plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict, RandomizedSearchCV, learning_curve
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score, precision_recall_curve, fbeta_score, make_scorer
from sklearn.calibration import calibration_curve
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from scipy.stats import wilcoxon
import joblib, os
from datetime import datetime
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
print('✅ 환경 설정 완료')

In [ ]:
DATA_DIR = '../data'
PROCESSED_DIR = os.path.join(DATA_DIR, 'processed')
os.makedirs(PROCESSED_DIR, exist_ok=True)
INPUT_FILE = os.path.join(DATA_DIR, 'features', 'domain_based_features_완전판.csv')
OUTPUT_PREFIX = '발표_Part3_v3'

## 1. 3-Way Split (Train 60% / Val 20% / Test 20%)

In [ ]:
df = pd.read_csv(INPUT_FILE, encoding='utf-8')
TARGET_COL = '모형개발용Performance(향후1년내부도여부)'
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=RANDOM_STATE)
print(f'Train: {len(X_train):,} | Val: {len(X_val):,} | Test: {len(X_test):,}')
print('⚠️ Test는 최종 평가까지 사용 안 함!')

## 2. 전처리 + ⭐ Winsorizer 실험 (Priority 3)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class InfiniteHandler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X): return X.replace([np.inf, -np.inf], np.nan)

class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, eps=1e-10): self.eps = eps
    def fit(self, X, y=None): return self
    def transform(self, X):
        X_c = X.copy()
        for c in X_c.columns:
            if (X_c[c] >= 0).all(): X_c[c] = np.log1p(X_c[c] + self.eps)
        return X_c

class Winsorizer(BaseEstimator, TransformerMixin):
    def __init__(self, l=0.005, u=0.995): self.l, self.u, self.b = l, u, {}
    def fit(self, X, y=None):
        for c in X.columns: self.b[c] = (X[c].quantile(self.l), X[c].quantile(self.u))
        return self
    def transform(self, X):
        X_c = X.copy()
        for c in X_c.columns: X_c[c] = X_c[c].clip(*self.b[c])
        return X_c

In [ ]:
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

def create_pipeline(clf, wins=False, resamp=None):
    s = [('inf', InfiniteHandler()), ('imp', SimpleImputer(strategy='median')), ('log', LogTransformer())]
    if wins: s.append(('wins', Winsorizer()))
    s.append(('scaler', RobustScaler()))
    s.append(('resamp', SMOTE(sampling_strategy=0.2, random_state=RANDOM_STATE) if resamp else 'passthrough'))
    s.append(('clf', clf))
    return ImbPipeline(s)

lgbm_t = lgb.LGBMClassifier(n_estimators=100, max_depth=5, learning_rate=0.05, scale_pos_weight=scale_pos_weight, random_state=RANDOM_STATE, verbose=-1)
print('Winsorizer 실험...')
wins_r = {}
for w in [False, True]:
    p = create_pipeline(lgbm_t, wins=w)
    p.fit(X_train, y_train)
    pr = average_precision_score(y_val, p.predict_proba(X_val)[:, 1])
    wins_r[w] = pr
    print(f'Wins={w}: PR-AUC={pr:.4f}')
USE_WINSORIZER = wins_r[True] > wins_r[False]
print(f'✅ Winsorizer={USE_WINSORIZER}')

## 3. 리샘플링 전략 (SMOTE vs Class Weight)

In [ ]:
lgbm_b = lgb.LGBMClassifier(n_estimators=100, max_depth=5, learning_rate=0.05, random_state=RANDOM_STATE, verbose=-1)
strat_a = {}
for v in ['smote', 'borderline', 'smote_tomek']:
    p = create_pipeline(lgbm_b, wins=USE_WINSORIZER, resamp=v)
    p.fit(X_train, y_train)
    pr = average_precision_score(y_val, p.predict_proba(X_val)[:, 1])
    strat_a[v] = pr
    print(f'{v}: {pr:.4f}')

lgbm_w = lgb.LGBMClassifier(n_estimators=100, max_depth=5, learning_rate=0.05, scale_pos_weight=scale_pos_weight, random_state=RANDOM_STATE, verbose=-1)
p_b = create_pipeline(lgbm_w, wins=USE_WINSORIZER)
p_b.fit(X_train, y_train)
strat_b_pr = average_precision_score(y_val, p_b.predict_proba(X_val)[:, 1])
print(f'Class Weight: {strat_b_pr:.4f}')

best_smote = max(strat_a, key=strat_a.get)
if strat_b_pr > strat_a[best_smote]:
    selected_strategy = 'Class Weight'
    selected_resampler = None
else:
    selected_strategy = f'SMOTE ({best_smote})'
    selected_resampler = best_smote
print(f'✅ 선택: {selected_strategy}')

## 4. ⭐ AutoML (Priority 1 - n_iter=200)

In [ ]:
lgbm_grid = {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7], 'learning_rate': [0.01, 0.05, 0.1], 'num_leaves': [31, 63, 127], 'min_child_samples': [20, 50, 100]}
xgb_grid = {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7], 'learning_rate': [0.01, 0.05, 0.1], 'min_child_weight': [1, 3, 5], 'subsample': [0.8, 0.9, 1.0]}
cb_grid = {'iterations': [100, 200, 300], 'depth': [4, 6, 8], 'learning_rate': [0.01, 0.05, 0.1], 'l2_leaf_reg': [1, 3, 5]}
lr_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2'], 'solver': ['liblinear']}

if selected_resampler is None:
    for g in [lgbm_grid, xgb_grid, cb_grid]: g['scale_pos_weight'] = [scale_pos_weight]
    lr_grid['class_weight'] = ['balanced']
else:
    for g in [lgbm_grid, xgb_grid, cb_grid]: g['scale_pos_weight'] = [1]
    lr_grid['class_weight'] = [None]

models = {
    'LightGBM': (lgb.LGBMClassifier(random_state=RANDOM_STATE, verbose=-1), lgbm_grid),
    'XGBoost': (xgb.XGBClassifier(random_state=RANDOM_STATE, eval_metric='logloss'), xgb_grid),
    'CatBoost': (CatBoostClassifier(random_state=RANDOM_STATE, verbose=0), cb_grid),
    'LogisticRegression': (LogisticRegression(random_state=RANDOM_STATE, max_iter=1000), lr_grid)
}

tuning_results = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scorer = make_scorer(average_precision_score, needs_proba=True)

print('AutoML 시작 (n_iter=200)...')
for name, (model, grid) in models.items():
    pipe = create_pipeline(model, wins=USE_WINSORIZER, resamp=selected_resampler)
    pipe_grid = {f'clf__{k}': v for k, v in grid.items()}
    search = RandomizedSearchCV(pipe, pipe_grid, n_iter=200, scoring=scorer, cv=cv, n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
    search.fit(X_train, y_train)
    tuning_results[name] = {'best_estimator': search.best_estimator_, 'best_cv_score': search.best_score_, 'best_params': search.best_params_}
    print(f'{name}: CV PR-AUC={search.best_score_:.4f}')
print('✅ AutoML 완료')

## 5. ⭐ 모델 선택 (Priority 1 - Ensemble + Statistical Test)

In [ ]:
# Validation 평가
val_results = {}
for name, res in tuning_results.items():
    pr = average_precision_score(y_val, res['best_estimator'].predict_proba(X_val)[:, 1])
    val_results[name] = pr
    print(f'{name}: Val PR-AUC={pr:.4f}')

best_single_name = max(val_results, key=val_results.get)
best_single = tuning_results[best_single_name]['best_estimator']
best_single_pr = val_results[best_single_name]
print(f'\nBest Single: {best_single_name} ({best_single_pr:.4f})')

In [ ]:
# Voting Ensemble (상위 3개)
sorted_models = sorted(val_results.items(), key=lambda x: x[1], reverse=True)[:3]
estimators = [(name, tuning_results[name]['best_estimator']) for name, _ in sorted_models]
voting = VotingClassifier(estimators=estimators, voting='soft')
voting.fit(X_train, y_train)
voting_pr = average_precision_score(y_val, voting.predict_proba(X_val)[:, 1])
print(f'Voting Ensemble: Val PR-AUC={voting_pr:.4f}')
print(f'\n성능 차이: {voting_pr - best_single_pr:+.4f}')

In [ ]:
# Statistical Significance Test
print('\n통계적 유의성 검정 (Wilcoxon)...')
cv_single = cross_val_score(best_single, X_train, y_train, cv=5, scoring=scorer)
cv_voting = cross_val_score(voting, X_train, y_train, cv=5, scoring=scorer)
stat, pval = wilcoxon(cv_voting, cv_single)
print(f'p-value: {pval:.4f}')

if voting_pr > best_single_pr and pval < 0.05:
    final_model = voting
    final_name = 'VotingEnsemble'
    reason = f'Ensemble이 {voting_pr - best_single_pr:.4f} 더 우수 (p={pval:.4f} < 0.05)'
else:
    final_model = best_single
    final_name = best_single_name
    reason = f'Single 선택 (차이 미미 또는 p={pval:.4f} >= 0.05)'

print(f'\n✅ 최종 모델: {final_name}')
print(f'   이유: {reason}')

## 6. ⭐ 임계값 최적화 (Priority 2 - Validation + CV)

In [ ]:
y_val_prob_final = final_model.predict_proba(X_val)[:, 1]
prec_v, rec_v, thr_v = precision_recall_curve(y_val, y_val_prob_final)

# F2-Score 최적
beta = 2
f2_scores = (1 + beta**2) * (prec_v[:-1] * rec_v[:-1]) / (beta**2 * prec_v[:-1] + rec_v[:-1] + 1e-10)
f2_opt_idx = np.argmax(f2_scores)
f2_opt_thr = thr_v[f2_opt_idx]
print(f'F2 최적 임계값 (Val): {f2_opt_thr:.4f}')

# Recall 80%
idx_80 = np.where(rec_v[:-1] >= 0.80)[0]
if len(idx_80) > 0:
    r80_idx = idx_80[np.argmax(prec_v[:-1][idx_80])]
    r80_thr = thr_v[r80_idx]
    print(f'Recall 80% 임계값 (Val): {r80_thr:.4f}')
else:
    r80_thr = f2_opt_thr
    print('Recall 80% 불가, F2 사용')

### 6.1 ⭐ CV 기반 임계값 검증 (Priority 2)

In [ ]:
# CV 예측
y_train_prob_cv = cross_val_predict(final_model, X_train, y_train, cv=5, method='predict_proba')[:, 1]
prec_cv, rec_cv, thr_cv = precision_recall_curve(y_train, y_train_prob_cv)

f2_cv = (1 + beta**2) * (prec_cv[:-1] * rec_cv[:-1]) / (beta**2 * prec_cv[:-1] + rec_cv[:-1] + 1e-10)
f2_cv_thr = thr_cv[np.argmax(f2_cv)]

idx_cv_80 = np.where(rec_cv[:-1] >= 0.80)[0]
if len(idx_cv_80) > 0:
    r80_cv_thr = thr_cv[idx_cv_80[np.argmax(prec_cv[:-1][idx_cv_80])]]
else:
    r80_cv_thr = f2_cv_thr

print(f'CV F2 임계값: {f2_cv_thr:.4f}')
print(f'CV Recall 80% 임계값: {r80_cv_thr:.4f}')

# 평균
selected_threshold = (r80_thr + r80_cv_thr) / 2
print(f'\n✅ 최종 임계값: {selected_threshold:.4f} (Val+CV 평균)')

## 7. Traffic Light 시스템

In [ ]:
# Red: Recall 80%
red_threshold = r80_thr

# Yellow: Recall 95%
idx_95 = np.where(rec_v[:-1] >= 0.95)[0]
if len(idx_95) > 0:
    yellow_threshold = thr_v[idx_95[np.argmax(prec_v[:-1][idx_95])]]
else:
    yellow_threshold = thr_v[-1] if len(thr_v) > 0 else 0.01

print(f'Red >= {red_threshold:.4f}')
print(f'Yellow >= {yellow_threshold:.4f}')

def assign_traffic(prob, red, yellow):
    if prob >= red: return 'Red'
    elif prob >= yellow: return 'Yellow'
    else: return 'Green'

# Val 평가
traffic_val = pd.Series(y_val_prob_final).apply(lambda x: assign_traffic(x, red_threshold, yellow_threshold))
for grade in ['Red', 'Yellow', 'Green']:
    mask = (traffic_val == grade)
    cnt = mask.sum()
    bk = y_val.values[mask].sum()
    print(f'{grade}: {cnt} 기업, {bk} 부도')

## 8. ⭐ Test Set 최종 평가 (Priority 2 - 시각화)

### 8.1 Test Set 예측

In [ ]:
y_test_prob = final_model.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_prob >= selected_threshold).astype(int)

test_pr_auc = average_precision_score(y_test, y_test_prob)
test_roc_auc = roc_auc_score(y_test, y_test_prob)
test_f2 = fbeta_score(y_test, y_test_pred, beta=2)

tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
test_recall = tp / (tp + fn)
test_prec = tp / (tp + fp)

print('='*70)
print('🎯 Test Set 최종 평가')
print('='*70)
print(f'PR-AUC: {test_pr_auc:.4f}')
print(f'ROC-AUC: {test_roc_auc:.4f}')
print(f'F2-Score: {test_f2:.4f}')
print(f'Precision: {test_prec:.2%}')
print(f'Recall: {test_recall:.2%}')
print(f'\nConfusion Matrix:')
print(f'  TN: {tn:,}  |  FP: {fp:,}')
print(f'  FN: {fn:,}  |  TP: {tp:,}')
print('='*70)

### 8.2 ⭐ PR-AUC Curve 시각화 (Priority 2)

In [ ]:
prec_test, rec_test, thr_test = precision_recall_curve(y_test, y_test_prob)

fig = go.Figure()
fig.add_trace(go.Scatter(x=rec_test, y=prec_test, mode='lines', name=f'PR-AUC={test_pr_auc:.4f}', line=dict(width=2)))
fig.add_trace(go.Scatter(x=[0, 1], y=[y_test.mean(), y_test.mean()], mode='lines', name='Baseline', line=dict(dash='dash')))
fig.update_layout(title='Precision-Recall Curve (Test Set)', xaxis_title='Recall', yaxis_title='Precision', height=500)
fig.show()

### 8.3 ⭐ Confusion Matrix 시각화 (Priority 2)

In [ ]:
cm = confusion_matrix(y_test, y_test_pred)
fig = go.Figure(data=go.Heatmap(z=cm, x=['Pred 0', 'Pred 1'], y=['True 0', 'True 1'], text=cm, texttemplate='%{text}', colorscale='Blues'))
fig.update_layout(title=f'Confusion Matrix (Threshold={selected_threshold:.4f})', height=400)
fig.show()

## 9. ⭐ 모델 진단 (Priority 3)

### 9.1 ⭐ Calibration Check (Priority 3)

In [ ]:
prob_true, prob_pred = calibration_curve(y_val, y_val_prob_final, n_bins=10, strategy='quantile')

fig = go.Figure()
fig.add_trace(go.Scatter(x=prob_pred, y=prob_true, mode='markers+lines', name='Model', marker=dict(size=10)))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', name='Perfect', line=dict(dash='dash')))
fig.update_layout(title='Calibration Curve (Validation Set)', xaxis_title='Predicted Probability', yaxis_title='True Probability', height=500)
fig.show()
print('\n✅ Calibration: 대각선에 가까울수록 확률 예측 정확')

### 9.2 ⭐ Learning Curve (Priority 3)

In [ ]:
train_sizes, train_scores, val_scores = learning_curve(
    final_model, X_train, y_train,
    cv=5, scoring=scorer,
    train_sizes=np.linspace(0.1, 1.0, 10),
    n_jobs=-1, random_state=RANDOM_STATE
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=train_sizes, y=train_scores.mean(axis=1), mode='lines+markers', name='Train', error_y=dict(array=train_scores.std(axis=1))))
fig.add_trace(go.Scatter(x=train_sizes, y=val_scores.mean(axis=1), mode='lines+markers', name='CV', error_y=dict(array=val_scores.std(axis=1))))
fig.update_layout(title='Learning Curve', xaxis_title='Training Size', yaxis_title='PR-AUC', height=500)
fig.show()
print('\n✅ Learning Curve: Train과 CV 격차가 작으면 과적합 없음')

## 10. ⭐ Cumulative Gains Curve (Priority 2)

In [ ]:
# Test Set Cumulative Gains
df_gains = pd.DataFrame({'prob': y_test_prob, 'y': y_test.values})
df_gains = df_gains.sort_values('prob', ascending=False).reset_index(drop=True)
df_gains['cum_y'] = df_gains['y'].cumsum()
df_gains['pct_captured'] = df_gains['cum_y'] / df_gains['y'].sum()
df_gains['pct_population'] = (df_gains.index + 1) / len(df_gains)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_gains['pct_population'], y=df_gains['pct_captured'], mode='lines', name='Model', line=dict(width=2)))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', name='Random', line=dict(dash='dash')))
fig.update_layout(title='Cumulative Gains Curve (Test Set)', xaxis_title='% Population', yaxis_title='% Bankruptcies Captured', xaxis_tickformat='.0%', yaxis_tickformat='.0%', height=500)
fig.show()

# 상위 10% 포착률
top10_captured = df_gains[df_gains['pct_population'] <= 0.1]['pct_captured'].iloc[-1]
print(f'\n✅ 상위 10% 기업에서 부도의 {top10_captured:.1%} 포착')
print(f'   효율성: Random 대비 {top10_captured/0.1:.1f}배')

## 11. Feature Importance

In [ ]:
if final_name == 'VotingEnsemble':
    # Ensemble이면 첫 번째 모델 사용
    clf = final_model.estimators_[0].named_steps['clf']
else:
    clf = final_model.named_steps['clf']

if hasattr(clf, 'feature_importances_'):
    imp = clf.feature_importances_
    feat_imp = pd.DataFrame({'Feature': X_train.columns, 'Importance': imp}).sort_values('Importance', ascending=False).head(15)
    
    fig = go.Figure(go.Bar(x=feat_imp['Importance'], y=feat_imp['Feature'], orientation='h', marker=dict(color=feat_imp['Importance'], colorscale='Blues')))
    fig.update_layout(title=f'Top 15 Feature Importance ({final_name})', xaxis_title='Importance', yaxis_title='Feature', height=600, yaxis={'categoryorder':'total ascending'})
    fig.show()
else:
    print('Feature Importance 지원 안 함')

## 12. 모델 및 결과 저장

In [ ]:
save_files = {
    f'{OUTPUT_PREFIX}_최종모델.pkl': final_model,
    f'{OUTPUT_PREFIX}_임계값.pkl': {'selected': selected_threshold, 'red': red_threshold, 'yellow': yellow_threshold},
    f'{OUTPUT_PREFIX}_결과.pkl': {'model_name': final_name, 'test_pr_auc': test_pr_auc, 'test_recall': test_recall, 'test_f2': test_f2}
}

for fname, data in save_files.items():
    joblib.dump(data, os.path.join(PROCESSED_DIR, fname))
    print(f'✅ {fname}')

print(f'\n저장 위치: {PROCESSED_DIR}')

## 13. 최종 요약

### ✅ v3 개선사항 완료

**Priority 1:**  
- Ensemble vs Single 비교 (Wilcoxon Test)  
- n_iter=200 (기존 30→200)  

**Priority 2:**  
- Cumulative Gains Curve  
- PR-AUC/Confusion Matrix 시각화  
- CV 기반 임계값 검증  

**Priority 3:**  
- Winsorizer 실험  
- Calibration Check  
- Learning Curve  

### 🎯 핵심 성과

✅ **Data Leakage 완전 제거**  
✅ **Statistical Significance 검증**  
✅ **Robust 임계값 (Val + CV 평균)**  
✅ **비즈니스 임팩트 입증** (Cumulative Gains)  
✅ **모델 진단 완료** (Calibration, Learning Curve)  

---

**노트북 완료!** 🎉